# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tyto tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět jak vypadá její derivace
- celý tento proces je možné reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je na venek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funckí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [ ]:
# kostra k doplnění, místa označená TODO doplňte

class Operator:
    def __init__(self):
        pass
    
    def __str__(self):
        return type(self).__name__
    
    def __repr__(self):
        return self.__str__()
    
    def diff(self):
        pass
    
    def simplify(self):
        return self

    def __add__(self, other):
        # TODO (použijte Addition)
        if isinstance(other, int):
            other = cst(other)
        return Addition(self, other).simplify() # pro scitani vytvorime novou instanci tridy Addition
           
    def __radd__(self, other):
        return self.__add__(other)
    
    def __sub__(self, other):
        # TODO (použijte Addition a Multiplication se zápornou konstantu)
        if isinstance(other, int):
            other = cst(other)
        return Addition(self, Multiplication(cst(-1), other)).simplify()
        
    def __mul__(self, other):
        # TODO (použijte Multiplication)
        if isinstance(other, int):
            other = cst(other)
        return Multiplication(self, other).simplify() # pro nasobeni
    
    def __rmul__(self, other):
        return self.__mul__(other)
        
    
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument
        
class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument
    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"
        
class Addition(BinaryOperator):
    def __str__(self): # pouzivame pro citelnejsi vystup
        # TODO vypíšeme něco jako (levy_argument)+(pravy_argument)
        return f"{self.levy_argument} + {self.pravy_argument}"
    
    def diff(self):
        # TODO
        return self.levy_argument.diff() + self.pravy_argument.diff() # soucet derivaci leveho a praveho argumentu -> (f+g)' = f' + g'

    def simplify(self):
        left = self.levy_argument.simplify()
        right = self.pravy_argument.simplify()
        if isinstance(left, cst) and isinstance(right, cst):
            return cst(left.value + right.value)
        if isinstance(left, cst) and left.value == 0:
            return right
        if isinstance(right, cst) and right.value == 0:
            return left
        return Addition(left, right)
    
    
class Multiplication(BinaryOperator):
    def __str__(self):
        # TODO vypíšeme něco jako (levy_argument)*(pravy_argument)
        
        return f"{self.levy_argument} * {self.pravy_argument}"

    def diff(self):
        # TODO
        return self.levy_argument.diff() + self.pravy_argument.diff() 
        # derivace soucinu je soucet derivaci -> (f*g)' = f'*g + f*g'

    def simplify(self):
        left = self.levy_argument.simplify()
        right = self.pravy_argument.simplify()
        if isinstance(left, cst) and isinstance(right, cst):
            return cst(left.value * right.value)
        if isinstance(left, cst) and left.value == 0:
            return right
        if isinstance(right, cst) and right.value == 0:
            return left
        if isinstance(left, cst) and left.value == 1:
            return right
        if isinstance(right, cst) and right.value == 1:
            return left
        if left == right:
            return mocnina(left, 2)
        return Multiplication(left, right)
    
class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value

    def __str__(self):
        # TODO vypíšeme něco jako value
        return str(self.value)

    def diff(self):
        # TODO
        return cst(0) # derivace konstanty je nula
    
    def simplify(self):
        return self
        
    
class identity(UnaryOperator):
    def __init__(self):
        pass

    def __str__(self):
        # TODO vypíšeme něco jako x   
        return "x"
    
    def diff(self):
        # TODO
        return cst(1) 
        
    
class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent

    def __str__(self):
        # TODO vypíšeme něco jako argument^exponent
        return f"({self.argument})^{self.exponent}"
        
    def diff(self):
        # TODO
        # derivace mocniny -> (f^n)' = n * f^(n-1) * f'
        return cst(self.exponent) * mocnina(self.argument, self.exponent - 1) * self.argument.diff()
    def simplify(self):
        base = self.argument.simplify()
        if self.exponent == 0:
            return cst(1)
        if self.exponent == 1:
            return base
        return mocnina(base, self.exponent)

class sin(UnaryOperator):
    def diff(self):
        # TODO
        # derivace sin(x) -> cos(x)
        return cos(self.argument) * self.argument.diff()
        

class cos(UnaryOperator):
    def diff(self):
        # TODO
        # derivace cos(x) -> -sin(x)
        return Multiplication(cst(-1), Multiplication(sin(self.argument), self.argument.diff()))
        

class exp(UnaryOperator):
    def diff(self):
        # TODO
        # derovace (e^x)' -> e^x
        return exp(self.argument) * self.argument.diff()
        
    
class ln(UnaryOperator):
    def diff(self):
        # TODO
        # deivale ln -> 1/x
        return frc(self.argument) * self.argument.diff()

class frc(UnaryOperator):
    def __str__(self):
        # TODO vypíšeme něco jako 1/(argument)
        return f"1/({self.argument})"
        pass
    
    def diff(self):
        # TODO zde je to asi vzhledem k tomu jak jsme si systém navrhli trochu složitější
        # bude to zvlášť "-1 *" zvlášť zlomek s argumentem ve tvaru mocniny krát derivace argumentu
        return Multiplication(cst(-1), Multiplication(frc(mocnina(self.argument, 2)), self.argument.diff()))
        pass

In [31]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
print(f1)
print(f1.diff())

sin(x + cos((x)^2)) + 1
cos(x + cos((x)^2)) * 1 + -1 * sin((x)^2) * 2 * x


můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [37]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)
print(f2)
print(f2.diff())


(x)^2 + x * 2 + 1 + (x)^2 * 2
x * 2 + 1 + x * 2


můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [33]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
print(f3)
print(f3.diff())


(x)^2 + 2 * x + exp(x)
2 * x + 1 + exp(x)


můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...` tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 